# Analyze data of consumers

This notebook aims at loading and analyzing measurement data of Italian consumers, whose data are provided by the standard export format from e-Distribuzione.

The data shall be all stored in a parent folder that contains subfolders, one for each consumers.
Each subfolder shall start with an unique identified that defines the POD. For example:
```
parent_folder/IT000001_Name1_Surname1/...
parent_folder/IT000002_Name2_Surname2/...
```
Each subfolder shall contain multiple xlsx files named "ExportData_*.xlsx" that contain the measurement data of the consumer.
The notebook loads all of them and creates a single dataframe that contains all the data.

### Setup parameters and environment

In [ ]:
data_folder = "Dati"  # Folder containing the data
START_POD = "IT"  # Identifier of the POD name

In [ ]:
# activate the environment
using Pkg
Pkg.activate(".")

### Load all data in the folder

Import major libraries

In [ ]:
using CSV, DataFrames, Dates, Plots

Create auxiliary functions

In [ ]:
"""
    read_data(fp)

Reads the data from the file with path 'fp' and return a DataFrame
- rows are the days
- columns are the quarters of each day
"""
function read_data(fp)
    df_temp = CSV.read(
        fp,
        DataFrame;
        delim=";",
        header=false,
        skipto=2,
    )[:, 1:97]
    rename!(df_temp, ["Day"; ["$i" for i = 1:96]])
    # convert date string to date
    df_temp[!, 1] = Date.(df_temp[!, 1], "dd/mm/yyyy")
    # convert strings to float
    df_temp[!, 2:end] = parse.(Float64, replace.(df_temp[!, 2:end], "," => "."))
    return df_temp
end

"""
    time_from_index(q::Int)::Dates.Time

Convert a quarter 'q' to a time.
"""
function time_from_index(q::Int)::Dates.Time
    h = floor(Int, (q-1)/4)
    m = 15*((q-1)%4)
    return Dates.Time(h, m, 0)
end

"""
standardize_data(df, name)

Standardize the data in the DataFrame 'df' and return a DataFrame with the following columns:
- pod_name: the name of the user
- datetime: datetime of the measurement
- value: the value of the measurement
"""
function standardize_data(df, pod_name)
    df_stacked = stack(df, 2:97)
    df_stacked[!, :variable] = parse.(Int, df_stacked[!, :variable])
    df_stacked[!, :time] = time_from_index.(df_stacked[!, :variable])
    df_stacked[!, :datetime] = DateTime.(df_stacked[!, :Day] .+ df_stacked[!, :time])
    df_stacked[!, :name] .= pod_name
    return df_stacked[!, [:name, :datetime, :value]]
end

"""
    get_POD_name(fp)

Extract the name of the POD from the file path 'fp'.
Assumptions: the name of the POD starts with "IT" and no other file or directory starts with "IT".
"""
function get_POD_name(fp, START_POD=START_POD)
    dirname = split(fp, "\\$START_POD")[2]
    POD_code = split(dirname, "_")[1]
    return "IT$POD_code"
end

Load all files

In [ ]:
df_list_quarter = []

for (root, dirs, files) in walkdir(data_folder)
    for file in files
        if endswith(file, ".csv")
            if startswith(file, "ExportData_")
                fp = joinpath(root, file)  # get the full path of the file
                println(fp)
                pod_name = get_POD_name(fp)  # get the name of the POD
                df = read_data(fp)  # read the data
                df_st = standardize_data(df, pod_name)  # standardize the data
                push!(df_list_quarter, df_st)  # append the DataFrame to the list
            else
                println("Skipping $file : not supported")
            end
        end
    end
end

Merge all data

In [ ]:
# merge data
df_all_data_raw = vcat(df_list_quarter...)

# drop duplicates
grp = groupby(df_all_data_raw, [:name, :datetime])
df_all_data = combine(grp, :value => first => :value)

# add auxiliary columns
df_all_data[!, :hour] = Dates.hour.(df_all_data[!, :datetime])
df_all_data[!, :quarter] = Dates.quarter.(df_all_data[!, :datetime])
df_all_data[!, :month] = Dates.month.(df_all_data[!, :datetime])

# sort values
sort!(df_all_data, [:name, :datetime])

first(df_all_data, 5)

### Analyze data

Create simple statistics

In [ ]:
# make some relevant statistics
grp = groupby(df_all_data, :name)
stats_by_POD = combine(
    grp,
    :datetime => (x->Date(minimum(x))) => :start,
    :datetime => (x->Date(maximum(x))) => :end,
    :datetime => (x->convert(Dates.Day, Date(maximum(x))-Date(minimum(x))) + Day(1)) => :delta_extremes,
    :datetime => (x->length(unique(Date.(x)))) => :n_days,
    :value => (x->sum(x)/1000) => :sum_values,
)
stats_by_POD

Make an average hourly plot by consumer

In [ ]:
grp = groupby(df_all_data, [:name, :hour])
avg_hourly = combine(
    grp,
    :value => (x->sum(x)/length(x)) => :avg_value,
)
avg_hourly

Create a dataframe with the average hourly consumption of each consumer:
- columns represent different consumers
- rows represent different hours

In [ ]:
df_by_POD = unstack(avg_hourly, :name, :avg_value)
sort!(df_by_POD, :hour)  # sort by hour
df_by_POD = 4 .* df_by_POD[!, Not(:hour)]  # drop the hour column
df_by_POD

Make the plot of the average hourly consumption by consumer

In [ ]:
x = plot()
for col in names(df_by_POD)
    plot!(x, df_by_POD[!, col], label=col, legend=:right)
end
xlabel!("Hour (h)")
ylabel!("Average consumption (kW)")
display(x)

In [ ]:
ylims!((0, 1))